# Purpose 

We will be registered and deploying the 19 rental calc models in what follows 

## Why? 


### Why Register? 
For the purposes of monitoring, we must have entries for a model in the registry. This part is required, because we specify the meta data of the model, specifically: 
* The type of model: regression, classification, etc
* The target of the model: the name of the target column
* If training data is available, add the training dataset to the registered model. This will permit us to track feature drift.

### Why "Deploy"
A deployment in DataRobot is a location that will provide use with details about the deployment (whereever it lives - in this case a DataRobot codespace.  Details include monitoring statistics, and other relevant details.  This location MUST exist so that there is a place to which we can report back monitoring statistics.  


In what follows, I have mocked up how I might register and deloy all models related to rental calc model


In [1]:
import numpy as np
import datarobot as dr 
import time 
client = dr.Client() 
models = list(range(5, 100, 5))

## Grab or Create Existing External Prediction Environment

The prediction environment for an external model is just for tracking purposes.  It lets us know where the model lives.  

In [2]:
## grab or create an existing prediction environment
try:
    prediction_environemnt = [p for p in dr.PredictionEnvironment.list() if p.name == "Rental Calc Prediction Environemt"].pop()
except:
    prediction_environment = dr.PredictionEnvironment.create(
        "Rental Calc Prediction Environemt", 
        platform = "other", 
        description = "External Prediction Environment for Rental Calc Models.  This is currently a scheduled notebook ina  datarobot codespace" 
    ) 

## Register Each Model as an External Model

In [ ]:
## if you have training data, register it to datarobot 
# dataset = dr.Dataset.create_from_in_memory_data(df)

registerd_external_models = []
for quantile in models:
    quantile = str(quantile)
    ext_reg_model = dr.RegisteredModelVersion.create_for_external(
        name = f"quantile {quantile}", 
        target = {"type": "Regression", "name": "charges"},
        # datasets = {"trainingDataCatalogId": dataset.id}, 
        registered_model_name = f"external quantile {quantile}",
        registered_model_description=f"quantile {quantile} model that has been packaged with umbrella model"
    )
    time.sleep(5)  ## introducing sleep so we don't get rate limited by datarobot.
    registerd_external_models.append(ext_reg_model)


## "Deploy" each Registered Model

I use quotes only because this is creating an entry the DataRobot console, to which we will report back monitoring statistics, either by way of prediction jobs.

In [4]:
registerd_external_models

In [5]:
deployments = []
for registerd_external_model in registerd_external_models:
    deployment = dr.Deployment.create_from_registered_model_version(
        model_package_id = registerd_external_model.id,
        label=registerd_external_model.name,
        description=f"external model deployment for {registerd_external_model.name}",
        # default_prediction_server_id=PREDICTION_SERVER.id,
        # importance="HIGH",
        prediction_environment_id=prediction_environemnt.id 
    )
    ## current feature drift tracking is disabled.  You need to provide a training dataset at time of model registration for this to be availabe
    deployment.update_drift_tracking_settings(target_drift_enabled=True, feature_drift_enabled=False)
    # Enabling Accuracy
    deployment.update_association_id_settings(column_names=["ASSOCIATION_ID"], required_in_prediction_requests=False)
    # Enabling Challenger
    deployment.update_predictions_data_collection_settings(enabled=True)
    deployments.append(deployment)




Next up, we write the data to disk so that we can call it the scoring code so we know where to report stuff back to 

In [6]:
import yaml 
routing_conf = []
for deployment in deployments:
    data = dict(
        deployment_id = deployment.id ,
        model_id = deployment.model["id"],
        target_column = deployment.label, 
        target_type = "Regression", 
        url = f"https://app.datarobot.com/console-nextgen/deployments/{deployment.id}/overview"
    )
    routing_conf.append(data)
with open("deployment_details.yaml", "w") as f:
    f.write(yaml.dump(routing_conf))

## tag deployments

Tags for deployments just provide us with a  way to filter on the Console.  

In [7]:
MODEL_TAG = "Rental Calc Models"

In [8]:
with open("deployment_details.yaml", "r") as f:
    model_config = yaml.load(f, Loader = yaml.SafeLoader)

In [9]:
for model in model_config:
    dep_id = model["deployment_id"]
    try:
        client.post(f"deployments/{dep_id}/tags", data = {"name": "Umbrella Model", "value": MODEL_TAG})
    except Exception as e:
        print(e)